In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import time

ALL = False
NO_SMOTE = True

# 判断 processed 文件夹是否存在
import os
if not os.path.exists('../data/processed'):
    print("Creating processed data folder...")
    # 读取CSV文件
    train_data = pd.read_csv('../data/raw/trainSet_res_with_distances.csv', dtype={'msisdn': 'str'})
    train_labels = pd.read_csv('../data/raw/trainSet_ans.csv', dtype={'msisdn': 'str'})

    validation_data = pd.read_csv('../data/raw/validationSet_res_with_distances.csv', dtype={'msisdn': 'str'})


    # 遍历 groupby('msisdn') 的结果，对每个 msisdn 进行数据增强
    # ------
    from tqdm import tqdm
    import os
    import sys
    sys.path.append(os.path.join(os.path.dirname('./'), '../'))
    from utils.augmentation import Augmentation

    addition_train_data = []
    addition_train_labels = []

    times = 8
    ratio_range = 0.1
    pbar = tqdm(train_data.groupby('msisdn'))
    for msisdn, group in pbar:
        if msisdn == 0:
            continue
        # print(f"Augmenting msisdn {msisdn}")
        pbar.set_description(f"Augmenting msisdn {msisdn}")
        label = train_labels[train_labels['msisdn'] == msisdn].iloc[0]['is_sa']
        aug = Augmentation(group, label, 'msisdn', 'is_sa')
        # 对正负样本进行平衡 样本比 1:4
        if label == 1:
            res_df, res_labels = aug.times(ratio=ratio_range, times=times * 4, method='mask')
        else:
            res_df, res_labels = aug.times(ratio=ratio_range, times=times, method='mask')
        addition_train_data.append(res_df)
        addition_train_labels.append(res_labels)
    addition_train_data = pd.concat(addition_train_data)
    addition_train_labels = pd.concat(addition_train_labels)

    # 将新数据加入到train_data中
    train_data = pd.concat([train_data, addition_train_data], ignore_index=True).reset_index(drop=True)
    train_labels = pd.concat([train_labels, addition_train_labels], ignore_index=True).reset_index(drop=True)
    # 按照 msisdn, start_time 排序
    sort_start_time = time.time()
    train_data = train_data.sort_values(by=['msisdn', 'start_time']).reset_index(drop=True)
    train_labels = train_labels.sort_values(by=['msisdn']).reset_index(drop=True)
    print('sort time:', time.time() - sort_start_time)
    # ------------------

    # save
    print("Saving processed data...")
    os.makedirs('../data/processed', exist_ok=True)
    train_data.to_csv('../data/processed/train_data.csv', index=False)
    train_labels.to_csv('../data/processed/train_labels.csv', index=False)
    validation_data.to_csv('../data/processed/validation_data.csv', index=False)

else:
    print("Reading processed data...")
    train_data = pd.read_csv('../data/processed/train_data.csv', dtype={'msisdn': 'str'})
    train_labels = pd.read_csv('../data/processed/train_labels.csv', dtype={'msisdn': 'str'})

    # 读取验证集
    validation_data = pd.read_csv('../data/processed/validation_data.csv', dtype={'msisdn': 'str'})


# 转换时间格式
train_data['start_time'] = pd.to_datetime(train_data['start_time'], format='%Y%m%d%H%M%S')
train_data['end_time'] = pd.to_datetime(train_data['end_time'], format='%Y%m%d%H%M%S')
train_data['open_datetime'] = pd.to_datetime(train_data['open_datetime'], format='%Y%m%d%H%M%S')
train_data['update_time'] = pd.to_datetime(train_data['update_time'])
train_data['date'] = pd.to_datetime(train_data['date'])

validation_data['start_time'] = pd.to_datetime(validation_data['start_time'], format='%Y%m%d%H%M%S')
validation_data['end_time'] = pd.to_datetime(validation_data['end_time'], format='%Y%m%d%H%M%S')
validation_data['open_datetime'] = pd.to_datetime(validation_data['open_datetime'], format='%Y%m%d%H%M%S',errors='coerce')
validation_data['update_time'] = pd.to_datetime(validation_data['update_time'])
validation_data['date'] = pd.to_datetime(validation_data['date'])

Creating processed data folder...


Augmenting msisdn 1304080: 100%|██████████| 3836/3836 [00:56<00:00, 67.43it/s]


sort time: 4.4887542724609375
Saving processed data...


In [40]:
len(train_data), len(validation_data)

(7562327, 232250)

In [41]:
# 为每条记录添加start_time_diff，记录 start_time 与上一条记录的 start_time 之差 (单位：秒)
start_time_diff = train_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
# 将该列加入到数据集中
train_data['start_time_diff'] = start_time_diff.copy()
# time_diff_start2end = train_data.groupby('msisdn')['end_time'].diff().dt.total_seconds().fillna(0)
start_time_diff = validation_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
validation_data['start_time_diff'] = start_time_diff.copy()

In [42]:
train_labels

,msisdn,is_sa
0,1000176,0
1,1000176_1,0
2,1000176_2,0
3,1000176_3,0
4,1000176_4,0
...,...,...
49543,1304080_4,0
49544,1304080_5,0
49545,1304080_6,0
49546,1304080_7,0


数据特征处理

In [43]:
# 聚合特征
def aggregate_features(data):
    return data.groupby('msisdn').agg({
    'call_duration': [
        # ('call_duration_sum', 'sum'), 
        ('call_duration_mean', 'mean'), 
        ('call_duration_max', 'max'), 
        ('call_duration_std', 'std'),
        ('call_duration_quantile_25', lambda x: x.quantile(0.25)), 
        ('call_duration_quantile_50', lambda x: x.quantile(0.50)), 
        ('call_duration_quantile_75', lambda x: x.quantile(0.75))
    ],
    'cfee': [
        # ('cfee_sum', 'sum'),
        ('cfee_std', 'std'), 
        ('cfee_mean', 'mean'),
    ],
    'lfee': [
        # ('lfee_sum', 'sum'), 
        ('lfee_mean', 'mean'),
        ('lfee_std', 'std'),
    ],
    'hour': [
        ('hour_mean', 'mean'), 
        ('hour_std', 'std'), 
        # ('hour_max', 'max'), 
        ('hour_min', 'min'),
    ],
    'dayofweek': [
        ('dayofweek_std', 'std'), 
        ('magic_dayofweek', lambda x: x.value_counts().mean()), 
        # ('work_day_num', lambda x: x[x.isin([1,2,3,4,5])].count()), 
        # ('weekend_num', lambda x: x[x.isin([6,7])].count()),
        ('dayofweek_mode', lambda x: x.mode().values[0]),
        ('work_day_weekend_diff', lambda x: (x[x.isin([1,2,3,4,5])].count() - x[x.isin([6,7])].count()) / (x[x.isin([1,2,3,4,5])].count() + x[x.isin([6,7])].count())),
    ],
    # 'home_area_code': [
    #     ('home_area_code_nunique', 'nunique')
    # ],
    'visit_area_code': [
        ('visit_area_code_nunique', 'nunique'),
        ('times_not_at_home_area', lambda x: x[x != x.shift()].count())
    ],
    'called_home_code': [
        ('called_home_code_nunique', 'nunique'),
        ('called_diff_home_code', lambda x: x[x != x.shift()].count())
    ],
    'called_code': [
        # ('called_code_nunique', 'nunique')
        ('called_code_diff', lambda x: x[x != x.shift()].count())
    ],
    'open_datetime': [
        ('open_count', 'nunique')
    ],
    'other_party': [
        ('account_person_num', 'nunique'),
        ('called_diff_home_code', lambda x: x[x != x.shift()].count())
    ],
    'a_serv_type': [
        # ('call_num', lambda x: x[x.isin([1, 3])].count()), 
        # ('called_num', lambda x: x[x == 2].count()),
        ('call_called_normalized_diff', lambda x: (x[x.isin([1, 3])].count() - x[x == 2].count()) /  (x[x.isin([1, 3])].count() + x[x == 2].count())),
    ],
    'start_time_diff': [
        ('start_time_diff_mean', 'mean'), 
        ('start_time_diff_std', 'std'), 
        ('start_time_diff_max', 'max'), 
    ], 
    'distance': [
        # ('distance_sum', 'sum'), 
        ('distance_std', 'std'), 
        # ('distance_max', 'max'), 
        # ('distance_quantile_25', lambda x: x.quantile(0.25)), 
        ('distance_quantile_50', lambda x: x.quantile(0.50)), 
        ('distance_quantile_75', lambda x: x.quantile(0.75)),
    ]
})

train_features = aggregate_features(train_data)
validation_features = aggregate_features(validation_data)


train_features.columns = ['_'.join(col).strip() for col in train_features.columns.values]
validation_features.columns = ['_'.join(col).strip() for col in validation_features.columns.values]
train_features.columns = train_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
validation_features.columns = validation_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
# 重置索引
train_features = train_features.reset_index()
validation_features = validation_features.reset_index()

# 合并标签数据
train_features = train_features.merge(train_labels, on='msisdn', how='left')
# 打印结果
# train_features

In [44]:
y = train_features['is_sa']
X = train_features.drop(['msisdn', 'is_sa'], axis=1)
X_validation = validation_features.drop(['msisdn'], axis=1)

n_sample = y.shape[0]
n_pos_sample = y[y ==1].shape[0]
n_neg_sample = y[y == 0].shape[0]
print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('特征维数：', X.shape[1])

样本个数：49548; 正样本占45.41%; 负样本占54.59%
特征维数： 32


In [45]:
# TODO use all_X to impute
imputer = SimpleImputer(strategy='most_frequent')
X = imputer.fit_transform(X)

In [46]:
imputer2 = SimpleImputer(strategy='most_frequent')
X_validation = imputer2.fit_transform(X_validation)

In [47]:
from sklearn.utils import shuffle
if ALL:
    if not NO_SMOTE:
        smote = SMOTE(random_state=42)    # 处理过采样的方法
        X, y = smote.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42, shuffle=True)
    train_len = len(y_train) + len(y_test)
    test_len = 0
else:
    # X_train,X_test,y_train,y_test = train_test_split(X,y,stratify = y,test_size= 0.3,random_state=42, shuffle=True)

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.3,random_state=42, shuffle=False)
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    X_test, y_test = shuffle(X_test, y_test, random_state=42)

    if not NO_SMOTE:
        smote = SMOTE(random_state=42)    # 处理过采样的方法
        X_train, y_train = smote.fit_resample(X_train, y_train)
        print('通过SMOTE方法平衡正负样本后')
        n_sample = y_train.shape[0]
        n_pos_sample = y_train[y_train == 1].shape[0]
        n_neg_sample = y_train[y_train == 0].shape[0]
        print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                        n_pos_sample / n_sample,
                                                        n_neg_sample / n_sample))
        print('特征维数：', X.shape[1])
    train_len, test_len = len(y_train), len(y_test)

In [48]:
X_test.shape

(14865, 32)

In [49]:
print(y_test.value_counts())

is_sa
0    7618
1    7247
Name: count, dtype: int64


In [50]:
X.shape

(49548, 32)

In [51]:
columns = train_features.columns.tolist()
columns.remove('msisdn')

In [52]:
assert X_validation.shape[1] == X_train.shape[1]

In [53]:
# 拼接 X_train 和 y_train np.array 为 dataframe
train_set = np.c_[X_train, y_train]
train_set = pd.DataFrame(train_set, columns=columns)
test_set = np.c_[X_test, y_test]
test_set = pd.DataFrame(test_set, columns=columns)
valid_set = np.c_[X_validation, np.zeros(X_validation.shape[0])]
valid_set = pd.DataFrame(valid_set, columns=columns)
valid_set['is_sa'] = -1

In [54]:
test_set.describe()

,call_duration_call_duration_mean,call_duration_call_duration_max,call_duration_call_duration_std,call_duration_call_duration_quantile_25,call_duration_call_duration_quantile_50,call_duration_call_duration_quantile_75,cfee_cfee_std,cfee_cfee_mean,lfee_lfee_mean,lfee_lfee_std,...,other_party_account_person_num,other_party_called_diff_home_code,a_serv_type_call_called_normalized_diff,start_time_diff_start_time_diff_mean,start_time_diff_start_time_diff_std,start_time_diff_start_time_diff_max,distance_distance_std,distance_distance_quantile_50,distance_distance_quantile_75,is_sa
count,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,...,14865.000000,14865.000000,14865.000000,14865.000000,14865.000000,1.486500e+04,1.486500e+04,1.486500e+04,1.486500e+04,14865.000000
mean,97.988029,985.518466,154.159790,22.754137,45.929297,103.571292,5.623005,3.246198,0.110847,0.290547,...,58.205045,113.752439,0.238326,19732.396478,39478.686655,1.921543e+05,2.955914e+05,2.934924e+05,4.272069e+05,0.487521
std,71.150893,945.675846,139.273917,11.219858,27.601590,86.155281,14.984967,8.691208,1.741936,2.933299,...,112.075480,168.083673,0.504225,22539.209496,42832.564146,1.802515e+05,2.884360e+05,5.278193e+05,6.246738e+05,0.499861
min,7.333333,9.000000,0.707107,3.000000,7.000000,9.000000,0.000000,0.000000,-18.783784,0.000000,...,1.000000,1.000000,-1.000000,0.000000,63.376672,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,53.280000,360.000000,63.745553,16.000000,29.000000,57.000000,0.000000,0.000000,0.000000,0.000000,...,13.000000,22.000000,-0.096296,4695.275862,11860.707583,7.175800e+04,2.483351e+04,0.000000e+00,0.000000e+00,0.000000
50%,80.128000,717.000000,113.993667,20.000000,39.000000,83.000000,0.000000,0.000000,0.000000,0.000000,...,27.000000,50.000000,0.179856,12327.373737,25562.072900,1.457680e+05,2.177537e+05,0.000000e+00,7.919600e+04,0.000000
75%,118.745614,1331.000000,198.638521,27.000000,55.000000,122.250000,0.000000,0.000000,0.000000,0.000000,...,57.000000,144.000000,0.662651,26937.266667,49914.769833,2.500120e+05,4.894950e+05,3.631260e+05,7.317870e+05,1.000000
max,670.492308,7201.000000,1465.065927,163.250000,333.000000,1044.000000,131.090412,64.583333,29.282051,44.402101,...,1294.000000,2411.000000,1.000000,402011.000000,692876.845683,1.202073e+06,2.047526e+06,3.831849e+06,3.831849e+06,1.000000


In [55]:
all_set = pd.concat([train_set, test_set, valid_set], axis=0).reset_index(drop=True)
labeled_data_len = train_set.shape[0] + test_set.shape[0]

In [56]:
test_set.shape, train_set.shape, valid_set.shape, all_set.shape

((14865, 33), (34683, 33), (1278, 33), (50826, 33))

In [57]:
labeled_set, valid_set = all_set.iloc[:labeled_data_len].copy(), all_set.iloc[labeled_data_len:].copy()
labeled_set.reset_index(drop=True, inplace=True)
valid_set.reset_index(drop=True, inplace=True)
# 有一些值在SMOTE后对数变换后为 NaN，需要删除这些数据
print(labeled_set.isnull().sum().sum())
labeled_set = labeled_set.dropna()
print(labeled_set.isnull().sum().sum())
assert valid_set.shape[0] == validation_features.shape[0]

0
0


In [58]:
# 重新划分训练集和测试集
if not ALL:
    train_set, test_set = labeled_set.iloc[:train_len].copy(), labeled_set.iloc[train_len:].copy()
    train_set.reset_index(drop=True, inplace=True)
    test_set.reset_index(drop=True, inplace=True)
    # # 从 test_set 和 y_test 中删除 数据增强获得的数据 无法实现
    # assert y.shape[0] == train_features.shape[0]
    # assert labeled_set.shape[0] == y.shape[0]
    # id_col = train_features['msisdn']
    # train_set_ids = id_col.iloc[train_set.index]
    # test_set_ids = id_col.iloc[test_set.index]

In [59]:
# 使用 autogluon 训练
from autogluon.tabular import TabularPredictor
# 输入数据X_train, y_train
if not ALL:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='medium_quality')
    # model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='best_quality', time_limit=3600)
else:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(labeled_set, presets='best_quality', time_limit=3600)

No path specified. Models will be saved in: "AutogluonModels\ag-20240719_070432"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240719_070432"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
CPU Count:          12
Memory Avail:       13.94 GB / 31.90 GB (43.7%)
Disk Space Avail:   489.23 GB / 3726.01 GB (13.1%)
Train Data Rows:    34683
Train Data Columns: 32
Label Column:       is_sa
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    14266.83 MB
	Train Data (Original)  Memory Usage: 8.47 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set 

In [60]:
if not ALL:
    print(model.evaluate(test_set))

{'f1': 0.8061208139721199, 'accuracy': 0.8372014799865456, 'balanced_accuracy': 0.833719807578827, 'mcc': 0.69845769825421, 'roc_auc': 0.9380124720405577, 'precision': 0.9610315186246419, 'recall': 0.6942182972264386}


In [61]:
feature_importance = model.feature_importance(test_set if not ALL else labeled_set)
print(feature_importance)
feature_importance

Computing feature importance via permutation shuffling for 32 features using 5000 rows with 5 shuffle sets...
	27.8s	= Expected runtime (5.56s per shuffle set)
	7.35s	= Actual runtime (Completed 5 of 5 shuffle sets)


                                           importance    stddev   p_value  n  \
other_party_called_diff_home_code            0.040345  0.003663  0.000008  5   
a_serv_type_call_called_normalized_diff      0.030570  0.006706  0.000261  5   
called_home_code_called_home_code_nunique    0.015929  0.004433  0.000651  5   
start_time_diff_start_time_diff_max          0.012437  0.001979  0.000074  5   
other_party_account_person_num               0.011782  0.003511  0.000843  5   
hour_hour_std                                0.011253  0.004238  0.002016  5   
start_time_diff_start_time_diff_std          0.010918  0.001965  0.000121  5   
called_code_called_code_diff                 0.008415  0.001148  0.000040  5   
hour_hour_min                                0.006288  0.003068  0.005080  5   
dayofweek_dayofweek_std                      0.005888  0.002115  0.001695  5   
distance_distance_std                        0.002561  0.002273  0.032709  5   
called_home_code_called_diff_home_code  

,importance,stddev,p_value,n,p99_high,p99_low
other_party_called_diff_home_code,0.040345,0.003663,0.000008,5,0.047887,0.032803
a_serv_type_call_called_normalized_diff,0.030570,0.006706,0.000261,5,0.044378,0.016762
called_home_code_called_home_code_nunique,0.015929,0.004433,0.000651,5,0.025056,0.006802
start_time_diff_start_time_diff_max,0.012437,0.001979,0.000074,5,0.016512,0.008361
other_party_account_person_num,0.011782,0.003511,0.000843,5,0.019010,0.004554
hour_hour_std,0.011253,0.004238,0.002016,5,0.019978,0.002528
start_time_diff_start_time_diff_std,0.010918,0.001965,0.000121,5,0.014965,0.006872
called_code_called_code_diff,0.008415,0.001148,0.000040,5,0.010778,0.006052
hour_hour_min,0.006288,0.003068,0.005080,5,0.012604,-0.000029
dayofweek_dayofweek_std,0.005888,0.002115,0.001695,5,0.010242,0.001534


In [62]:
# leaderboard
if not ALL:
    leaderboard = model.leaderboard(test_set, silent=True)
    print(leaderboard)
else:
    leaderboard = model.leaderboard(labeled_set, silent=True)
    print(leaderboard)
leaderboard

                  model  score_test  score_val eval_metric  pred_time_test  \
0            LightGBMXT    0.836691   0.999089          f1        0.093228   
1              LightGBM    0.826138   0.999089          f1        0.097000   
2               XGBoost    0.820670   0.997724          f1        0.077329   
3        NeuralNetTorch    0.818939   0.998180          f1        0.133231   
4              CatBoost    0.810794   0.999090          f1        0.016004   
5   WeightedEnsemble_L2    0.806121   0.999545          f1        0.171129   
6         LightGBMLarge    0.806088   0.997724          f1        0.051541   
7       NeuralNetFastAI    0.799780   0.998635          f1        0.213573   
8      RandomForestEntr    0.794472   0.998633          f1        0.141695   
9      RandomForestGini    0.786329   0.998633          f1        0.161916   
10       ExtraTreesEntr    0.770882   0.998633          f1        0.186390   
11       ExtraTreesGini    0.762672   0.998633          f1      

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.836691,0.999089,f1,0.093228,0.036394,3.972994,0.093228,0.036394,3.972994,1,True,3
1,LightGBM,0.826138,0.999089,f1,0.097000,0.023077,3.355544,0.097000,0.023077,3.355544,1,True,4
2,XGBoost,0.820670,0.997724,f1,0.077329,0.020066,2.148064,0.077329,0.020066,2.148064,1,True,11
3,NeuralNetTorch,0.818939,0.998180,f1,0.133231,0.040927,56.461948,0.133231,0.040927,56.461948,1,True,12
4,CatBoost,0.810794,0.999090,f1,0.016004,0.009433,20.200444,0.016004,0.009433,20.200444,1,True,7
5,WeightedEnsemble_L2,0.806121,0.999545,f1,0.171129,0.128794,25.206867,0.003002,0.005000,3.604374,2,True,14
6,LightGBMLarge,0.806088,0.997724,f1,0.051541,0.017001,2.700315,0.051541,0.017001,2.700315,1,True,13
7,NeuralNetFastAI,0.799780,0.998635,f1,0.213573,0.044139,35.753077,0.213573,0.044139,35.753077,1,True,10
8,RandomForestEntr,0.794472,0.998633,f1,0.141695,0.083369,4.184905,0.141695,0.083369,4.184905,1,True,6
9,RandomForestGini,0.786329,0.998633,f1,0.161916,0.068759,3.947740,0.161916,0.068759,3.947740,1,True,5


In [63]:
# 在testset 上计算指标
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

if not ALL:
    y_pred = model.predict(test_set)
    y_true = test_set['is_sa']
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.77      0.97      0.86      7618
         1.0       0.96      0.69      0.81      7247

    accuracy                           0.84     14865
   macro avg       0.87      0.83      0.83     14865
weighted avg       0.86      0.84      0.83     14865

[[7414  204]
 [2216 5031]]


In [64]:
valid_set

,call_duration_call_duration_mean,call_duration_call_duration_max,call_duration_call_duration_std,call_duration_call_duration_quantile_25,call_duration_call_duration_quantile_50,call_duration_call_duration_quantile_75,cfee_cfee_std,cfee_cfee_mean,lfee_lfee_mean,lfee_lfee_std,...,other_party_account_person_num,other_party_called_diff_home_code,a_serv_type_call_called_normalized_diff,start_time_diff_start_time_diff_mean,start_time_diff_start_time_diff_std,start_time_diff_start_time_diff_max,distance_distance_std,distance_distance_quantile_50,distance_distance_quantile_75,is_sa
0,60.308682,1700.0,112.715262,19.50,37.0,62.50,0.000000,0.00,0.0,0.0,...,69.0,249.0,-0.421222,4071.543408,8323.861851,51194.0,2.081493e+05,0.0,180274.0,-1.0
1,54.165854,362.0,56.854232,17.00,37.0,65.00,0.000000,0.00,0.0,0.0,...,72.0,174.0,-0.160976,6229.775610,10913.381674,51724.0,2.102743e+04,0.0,0.0,-1.0
2,89.500000,670.0,175.297814,12.50,37.5,49.00,0.000000,0.00,0.0,0.0,...,8.0,14.0,-0.333333,65299.611111,72201.374485,217652.0,0.000000e+00,0.0,0.0,-1.0
3,46.458515,399.0,58.811953,14.00,27.0,53.00,0.000000,0.00,0.0,0.0,...,27.0,155.0,0.362445,5491.794760,11149.929456,65807.0,0.000000e+00,231682.0,231682.0,-1.0
4,38.430556,246.0,46.076735,13.00,19.0,42.50,6.804617,3.75,0.0,0.0,...,44.0,51.0,0.500000,16717.666667,39100.045311,242220.0,1.696337e+05,29847.5,280421.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,156.426230,1804.0,318.203739,23.00,45.0,96.50,0.000000,0.00,0.0,0.0,...,55.0,144.0,0.136612,6796.404372,17418.083154,148767.0,7.154820e+04,0.0,0.0,-1.0
1274,52.897872,621.0,69.502307,19.00,34.0,54.00,0.000000,0.00,0.0,0.0,...,36.0,130.0,0.055319,5298.587234,13151.545900,144098.0,1.749472e+05,0.0,0.0,-1.0
1275,346.552448,4061.0,637.773302,34.00,96.0,389.50,0.000000,0.00,0.0,0.0,...,23.0,114.0,0.230769,8752.293706,15958.753297,72131.0,3.078249e+05,0.0,0.0,-1.0
1276,22.500000,44.0,30.405592,11.75,22.5,33.25,0.000000,0.00,0.0,0.0,...,2.0,2.0,0.000000,91577.000000,129509.435401,183154.0,1.117818e+06,790417.0,1185625.5,-1.0


In [65]:
# 预测
y_validation_pred = model.predict(valid_set.drop('is_sa', axis=1))

# 将预测结果与 msisdn 对应起来
validation_results = validation_features[['msisdn']].copy()
validation_results['is_sa'] = y_validation_pred.astype(int)

print(validation_results.describe())

# 保存结果到CSV文件
file_name = './valid_large_data.csv' if ALL else './valid_small_data.csv'
validation_results.to_csv(file_name, index=False)
print(file_name)

             is_sa
count  1278.000000
mean      0.166667
std       0.372824
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
./valid_small_data.csv
